# Evaluate nasnet
Evaluate nasnet using micronet challenge [scoring guidelines](https://micronet-challenge.github.io/scoring_and_submission.html).

### Imports

In [1]:
PATH = '/workspace/micronet_challenge_2019/'
import os
import sys
import argparse
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.autograd import Variable

os.chdir(PATH)
from conf import settings
from utils.model_utils import get_network
from utils.model_counting import count_Conv2d, count_NormalCell, count_ReductionCell
from utils.model_counting import countReLU, count_AdaptiveAvgPool2d, count_FullyConnected

### Parameters

In [2]:
def parse_args(args):
    """
    Takes as input a string of args
    """
    parser = argparse.ArgumentParser()
    parser.add_argument('-net', type=str, required=True, help='net type')
    parser.add_argument('-gpu', type=bool, default=True, help='use gpu or not')
    parser.add_argument('-w', type=int, default=2, help='number of workers for dataloader')
    parser.add_argument('-b', type=int, default=32, help='batch size for dataloader')
    parser.add_argument('-s', type=bool, default=True, help='whether shuffle the dataset')
    parser.add_argument('-warm', type=int, default=1, help='warm up training phase')
    parser.add_argument('-lr', type=float, default=0.1, help='initial learning rate')
    return parser.parse_args(args.split())
    

In [3]:
args = '-net nasnet -b 32 -lr 0.025'
args = parse_args(args)

### Load Model

In [4]:
net = get_network(args, use_gpu=args.gpu)

# load model from checkpoint
CHECKPOINT_PATH = PATH + 'checkpoint/nasnet-595-best.pth'
net.load_state_dict(torch.load(CHECKPOINT_PATH))

### Load Data

In [5]:
def get_transform_test():
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(settings.CIFAR100_TRAIN_MEAN, settings.CIFAR100_TRAIN_STD),
    ])
    return transform_test

def get_test_dataloader(mean, std, batch_size=16, num_workers=2, shuffle=True):
    """ return training dataloader
    Args:
        mean: mean of cifar100 test dataset
        std: std of cifar100 test dataset
        path: path to cifar100 test python dataset
        batch_size: dataloader batchsize
        num_workers: dataloader num_works
        shuffle: whether to shuffle 
    Returns: cifar100_test_loader:torch dataloader object
    """

    transform_test = get_transform_test()
    cifar100_test = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
    cifar100_test_loader = DataLoader(
        cifar100_test, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size)

    return cifar100_test_loader


# get the cifar100 test dataset
cifar100_test_loader = get_test_dataloader(
    settings.CIFAR100_TRAIN_MEAN,
    settings.CIFAR100_TRAIN_STD,
    num_workers=args.w,
    batch_size=args.b,
    shuffle=args.s
)

Files already downloaded and verified


## Test Accuracy
Accuracy should be above 80 percent.

In [6]:
net.eval()

correct = 0.0

for (images, labels) in cifar100_test_loader:
    images = Variable(images)
    labels = Variable(labels)

    images = images.cuda()
    labels = labels.cuda()

    outputs = net(images)
    _, preds = outputs.max(1)
    correct += preds.eq(labels).sum()

acc = correct.float() / len(cifar100_test_loader.dataset)

In [7]:
print('Accuracy for ' + args.net + ': ' + '{0:.4%}'.format(acc.item()) )

Accuracy for nasnet: 81.2800%


## Parameter Storage
The model performs no quantization, so we use freebie quantization and divide the total number of parameters by 2. 

In [8]:
total_params = sum(p.numel() for p in net.parameters())
print(args.net +  ' has ' + '{0}'.format(total_params) + ' total parameters.')

total_params_freebie = total_params / 2
benchmark_params = 36.5e6
parameter_score = total_params_freebie / benchmark_params
print(args.net + ' achieves a parameter score of ' + '{0:.4}'.format(parameter_score))

nasnet has 5221624 total parameters.
nasnet achieves a parameter score of 0.07153


## Math Operations
The model performs no quantization, so we use freebie quantization and divide the total number of multiplies by 2. 

### Stem
The stem for nasnet consists of one Conv2d followed by BachNorm2d Operation. We assume a [fused](https://tehnokv.com/posts/fusing-batchnorm-and-conv) conv + batchnorm layer

In [16]:
total_mults = total_adds = 0
x = torch.Tensor(1, 3, 32, 32).cuda()
conv_layer = net.stem[0]

# (1) Conv2D and (3) BatchNorm2d
flop_mults, flop_adds = count_Conv2d(conv_layer, x)
total_mults += flop_mults; total_adds += flop_adds
x = net.stem[0](x)
# assume fused conv layer at batch norm at inference
x = net.stem[1](x)
print('Total mults: ' + '{0}'.format(total_mults) + ' and adds: ' + '{0}'.format(total_adds) + ' for stem layer.')

Total mults: 1216512 and adds: 1171456 for stem layer.


### Cell Layers
Nasnet has several sequential and reduction layers.

### Normal Cell (Layers 0 - 3)

In [17]:
prev = None

for i in range(4):
    normalcell = net.cell_layers[i]
    flop_mults, flop_adds = count_NormalCell(normalcell, x, prev)
    total_mults += flop_mults; total_adds += flop_adds
    x, prev = normalcell((x, prev))

### Reduction Cell (Layer 4)

In [18]:
reductioncell = net.cell_layers[4]
flop_mults, flop_adds = count_ReductionCell(reductioncell, x, prev)
total_mults += flop_mults; total_adds += flop_adds
x, prev = reductioncell((x, prev))

### Normal Cell (Layers 5 - 8)

In [19]:
for i in range(5, 9):
    normalcell = net.cell_layers[i]
    flop_mults, flop_adds = count_NormalCell(normalcell, x, prev)
    total_mults += flop_mults; total_adds += flop_adds
    x, prev = normalcell((x, prev))

### Reduction Cell (Layer 9)

In [20]:
reductioncell = net.cell_layers[9]
flop_mults, flop_adds = count_ReductionCell(reductioncell, x, prev)
total_mults += flop_mults; total_adds += flop_adds
x, prev = reductioncell((x, prev))

### Normal Cell (Layers 10 - 13)

In [21]:
for i in range(10, 14):
    normalcell = net.cell_layers[i]
    flop_mults, flop_adds = count_NormalCell(normalcell, x, prev)
    total_mults += flop_mults; total_adds += flop_adds
    x, prev = normalcell((x, prev))

### Final Layers

In [22]:
# Relu
flop_mults, flop_adds = countReLU(x)
total_mults += flop_mults; total_adds += flop_adds
x = net.relu(x)

# Adaptive Average Pooling 2D
pool_layer = net.avg
flop_mults, flop_adds = count_AdaptiveAvgPool2d(pool_layer, x)
total_mults += flop_mults; total_adds += flop_adds
x = pool_layer(x)

# changing the view
x = x.view(x.size(0), -1)

# Fully connected layer
fc_layer = net.fc
flop_mults, flop_adds = count_FullyConnected(fc_layer, x)
total_mults += flop_mults; total_adds += flop_adds
x = fc_layer(x)

n_classes = 100
assert x.shape[1] == n_classes

### Calculate Score

In [23]:
total_mults_freebie = total_mults / 2
benchmark_ops = 10.49e9
ops_score = (total_mults_freebie + total_adds) / benchmark_ops
print('Total mults: ' + '{0}'.format(total_mults) + ' and adds: ' + '{0}'.format(total_adds) + ' for nasnet.')
print(args.net + ' achieves an ops score of ' + '{0:.4}'.format(ops_score))

Total mults: 662173600 and adds: 659313248 for nasnet.
nasnet achieves an ops score of 0.09441


### Final Score

In [24]:
print('Final Score: ' + '{0:.8}'.format(ops_score + parameter_score))

Final Score: 0.16594283
